[SDS-2.x, Scalable Data Engineering Science](https://lamastex.github.io/scalable-data-science/sds/2/x/)
=======================================================================================================

This is a 2019 augmentation and update of [Adam
Breindel](https://www.linkedin.com/in/adbreind)'s initial notebooks.

In [ ]:
'''Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

  

>     Using TensorFlow backend.
>     Downloading data from https://s3.amazonaws.com/text-datasets/nietzsche.txt
>
>      16384/600901 [..............................] - ETA: 0s
>      24576/600901 [>.............................] - ETA: 1s
>      57344/600901 [=>............................] - ETA: 1s
>     139264/600901 [=====>........................] - ETA: 0s
>     303104/600901 [==============>...............] - ETA: 0s
>     606208/600901 [==============================] - 0s 0us/step
>
>     614400/600901 [==============================] - 0s 0us/step
>     corpus length: 600901
>     total chars: 59
>     nb sequences: 200287
>     Vectorization...
>     Build model...
>     WARNING:tensorflow:From /databricks/python/local/lib/python2.7/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
>     Instructions for updating:
>     Colocations handled automatically by placer.
>
>     --------------------------------------------------
>     Iteration 1
>     WARNING:tensorflow:From /databricks/python/local/lib/python2.7/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
>     Instructions for updating:
>     Use tf.cast instead.
>     Epoch 1/1
>
>        128/200287 [..............................] - ETA: 16:02 - loss: 4.0839
>        256/200287 [..............................] - ETA: 9:16 - loss: 3.8942 
>        384/200287 [..............................] - ETA: 6:59 - loss: 3.9822
>        512/200287 [..............................] - ETA: 5:49 - loss: 3.8959
>        640/200287 [..............................] - ETA: 5:08 - loss: 3.8016
>        768/200287 [..............................] - ETA: 4:41 - loss: 3.6985
>        896/200287 [..............................] - ETA: 4:20 - loss: 3.6066
>       1024/200287 [..............................] - ETA: 4:05 - loss: 3.5509
>       1152/200287 [..............................] - ETA: 3:53 - loss: 3.4982
>       1280/200287 [..............................] - ETA: 3:44 - loss: 3.4572
>       1408/200287 [..............................] - ETA: 3:36 - loss: 3.4184
>       1536/200287 [..............................] - ETA: 3:30 - loss: 3.3912
>       1664/200287 [..............................] - ETA: 3:24 - loss: 3.3591
>       1792/200287 [..............................] - ETA: 3:19 - loss: 3.3321
>       1920/200287 [..............................] - ETA: 3:15 - loss: 3.3156
>       2048/200287 [..............................] - ETA: 3:12 - loss: 3.2956
>       2176/200287 [..............................] - ETA: 3:09 - loss: 3.2771
>       2304/200287 [..............................] - ETA: 3:06 - loss: 3.2685
>       2432/200287 [..............................] - ETA: 3:04 - loss: 3.2512
>       2560/200287 [..............................] - ETA: 3:01 - loss: 3.2443
>       2688/200287 [..............................] - ETA: 2:59 - loss: 3.2235
>       2816/200287 [..............................] - ETA: 2:57 - loss: 3.2178
>       2944/200287 [..............................] - ETA: 2:56 - loss: 3.2054
>       3072/200287 [..............................] - ETA: 2:54 - loss: 3.1919
>       3200/200287 [..............................] - ETA: 2:53 - loss: 3.1835
>       3328/200287 [..............................] - ETA: 2:51 - loss: 3.1701
>       3456/200287 [..............................] - ETA: 2:50 - loss: 3.1651
>       3584/200287 [..............................] - ETA: 2:49 - loss: 3.1694
>       3712/200287 [..............................] - ETA: 2:48 - loss: 3.1606
>       3840/200287 [..............................] - ETA: 2:47 - loss: 3.1624
>       3968/200287 [..............................] - ETA: 2:46 - loss: 3.1551
>       4096/200287 [..............................] - ETA: 2:45 - loss: 3.1463
>       4224/200287 [..............................] - ETA: 2:44 - loss: 3.1350
>       4352/200287 [..............................] - ETA: 2:43 - loss: 3.1332
>       4480/200287 [..............................] - ETA: 2:43 - loss: 3.1294
>       4608/200287 [..............................] - ETA: 2:42 - loss: 3.1231
>       4736/200287 [..............................] - ETA: 2:41 - loss: 3.1206
>       4864/200287 [..............................] - ETA: 2:40 - loss: 3.1167
>       4992/200287 [..............................] - ETA: 2:40 - loss: 3.1124
>       5120/200287 [..............................] - ETA: 2:39 - loss: 3.1045
>       5248/200287 [..............................] - ETA: 2:38 - loss: 3.1017
>       5376/200287 [..............................] - ETA: 2:38 - loss: 3.0951
>       5504/200287 [..............................] - ETA: 2:37 - loss: 3.0877
>       5632/200287 [..............................] - ETA: 2:37 - loss: 3.0805
>       5760/200287 [..............................] - ETA: 2:36 - loss: 3.0795
>       5888/200287 [..............................] - ETA: 2:35 - loss: 3.0719
>       6016/200287 [..............................] - ETA: 2:35 - loss: 3.0690
>       6144/200287 [..............................] - ETA: 2:34 - loss: 3.0648
>       6272/200287 [..............................] - ETA: 2:34 - loss: 3.0596
>       6400/200287 [..............................] - ETA: 2:34 - loss: 3.0539
>       6528/200287 [..............................] - ETA: 2:33 - loss: 3.0468
>       6656/200287 [..............................] - ETA: 2:33 - loss: 3.0399
>       6784/200287 [>.............................] - ETA: 2:33 - loss: 3.0347
>       6912/200287 [>.............................] - ETA: 2:32 - loss: 3.0260
>       7040/200287 [>.............................] - ETA: 2:32 - loss: 3.0204
>       7168/200287 [>.............................] - ETA: 2:32 - loss: 3.0162
>       7296/200287 [>.............................] - ETA: 2:31 - loss: 3.0065
>       7424/200287 [>.............................] - ETA: 2:31 - loss: 2.9983
>       7552/200287 [>.............................] - ETA: 2:31 - loss: 2.9908
>       7680/200287 [>.............................] - ETA: 2:30 - loss: 2.9868
>       7808/200287 [>.............................] - ETA: 2:30 - loss: 2.9825
>       7936/200287 [>.............................] - ETA: 2:30 - loss: 2.9790
>       8064/200287 [>.............................] - ETA: 2:29 - loss: 2.9735
>       8192/200287 [>.............................] - ETA: 2:29 - loss: 2.9676
>       8320/200287 [>.............................] - ETA: 2:29 - loss: 2.9644
>       8448/200287 [>.............................] - ETA: 2:28 - loss: 2.9606
>       8576/200287 [>.............................] - ETA: 2:28 - loss: 2.9558
>       8704/200287 [>.............................] - ETA: 2:28 - loss: 2.9543
>       8832/200287 [>.............................] - ETA: 2:28 - loss: 2.9486
>       8960/200287 [>.............................] - ETA: 2:27 - loss: 2.9429
>       9088/200287 [>.............................] - ETA: 2:27 - loss: 2.9384
>       9216/200287 [>.............................] - ETA: 2:27 - loss: 2.9334
>       9344/200287 [>.............................] - ETA: 2:27 - loss: 2.9272
>       9472/200287 [>.............................] - ETA: 2:26 - loss: 2.9243
>       9600/200287 [>.............................] - ETA: 2:26 - loss: 2.9207
>       9728/200287 [>.............................] - ETA: 2:26 - loss: 2.9161
>       9856/200287 [>.............................] - ETA: 2:26 - loss: 2.9117
>       9984/200287 [>.............................] - ETA: 2:25 - loss: 2.9099
>      10112/200287 [>.............................] - ETA: 2:25 - loss: 2.9037
>      10240/200287 [>.............................] - ETA: 2:25 - loss: 2.9005
>      10368/200287 [>.............................] - ETA: 2:25 - loss: 2.8964
>      10496/200287 [>.............................] - ETA: 2:24 - loss: 2.8934
>      10624/200287 [>.............................] - ETA: 2:24 - loss: 2.8901
>      10752/200287 [>.............................] - ETA: 2:24 - loss: 2.8876
>      10880/200287 [>.............................] - ETA: 2:24 - loss: 2.8810
>      11008/200287 [>.............................] - ETA: 2:24 - loss: 2.8751
>      11136/200287 [>.............................] - ETA: 2:23 - loss: 2.8703
>      11264/200287 [>.............................] - ETA: 2:23 - loss: 2.8664
>      11392/200287 [>.............................] - ETA: 2:23 - loss: 2.8611
>      11520/200287 [>.............................] - ETA: 2:23 - loss: 2.8578
>      11648/200287 [>.............................] - ETA: 2:23 - loss: 2.8537
>      11776/200287 [>.............................] - ETA: 2:23 - loss: 2.8474
>      11904/200287 [>.............................] - ETA: 2:23 - loss: 2.8470
>      12032/200287 [>.............................] - ETA: 2:22 - loss: 2.8435
>      12160/200287 [>.............................] - ETA: 2:22 - loss: 2.8400
>      12288/200287 [>.............................] - ETA: 2:22 - loss: 2.8385
>      12416/200287 [>.............................] - ETA: 2:22 - loss: 2.8354
>      12544/200287 [>.............................] - ETA: 2:22 - loss: 2.8327
>      12672/200287 [>.............................] - ETA: 2:22 - loss: 2.8309
>      12800/200287 [>.............................] - ETA: 2:21 - loss: 2.8279
>      12928/200287 [>.............................] - ETA: 2:21 - loss: 2.8250
>      13056/200287 [>.............................] - ETA: 2:21 - loss: 2.8211
>      13184/200287 [>.............................] - ETA: 2:21 - loss: 2.8202
>      13312/200287 [>.............................] - ETA: 2:21 - loss: 2.8199
>      13440/200287 [=>............................] - ETA: 2:20 - loss: 2.8197
>      13568/200287 [=>............................] - ETA: 2:20 - loss: 2.8174
>      13696/200287 [=>............................] - ETA: 2:20 - loss: 2.8153
>      13824/200287 [=>............................] - ETA: 2:20 - loss: 2.8107
>      13952/200287 [=>............................] - ETA: 2:20 - loss: 2.8079
>      14080/200287 [=>............................] - ETA: 2:20 - loss: 2.8054
>      14208/200287 [=>............................] - ETA: 2:20 - loss: 2.8020
>      14336/200287 [=>............................] - ETA: 2:19 - loss: 2.7991
>      14464/200287 [=>............................] - ETA: 2:19 - loss: 2.7960
>      14592/200287 [=>............................] - ETA: 2:19 - loss: 2.7918
>      14720/200287 [=>............................] - ETA: 2:19 - loss: 2.7887
>      14848/200287 [=>............................] - ETA: 2:19 - loss: 2.7872
>      14976/200287 [=>............................] - ETA: 2:19 - loss: 2.7852
>      15104/200287 [=>............................] - ETA: 2:18 - loss: 2.7820
>      15232/200287 [=>............................] - ETA: 2:18 - loss: 2.7792
>      15360/200287 [=>............................] - ETA: 2:18 - loss: 2.7762
>      15488/200287 [=>............................] - ETA: 2:18 - loss: 2.7728
>      15616/200287 [=>............................] - ETA: 2:18 - loss: 2.7695
>      15744/200287 [=>............................] - ETA: 2:18 - loss: 2.7679
>      15872/200287 [=>............................] - ETA: 2:18 - loss: 2.7653
>      16000/200287 [=>............................] - ETA: 2:17 - loss: 2.7642
>      16128/200287 [=>............................] - ETA: 2:17 - loss: 2.7608
>      16256/200287 [=>............................] - ETA: 2:17 - loss: 2.7569
>      16384/200287 [=>............................] - ETA: 2:17 - loss: 2.7540
>      16512/200287 [=>............................] - ETA: 2:17 - loss: 2.7515
>      16640/200287 [=>............................] - ETA: 2:17 - loss: 2.7500
>      16768/200287 [=>............................] - ETA: 2:17 - loss: 2.7477
>      16896/200287 [=>............................] - ETA: 2:16 - loss: 2.7449
>      17024/200287 [=>............................] - ETA: 2:16 - loss: 2.7428
>      17152/200287 [=>............................] - ETA: 2:16 - loss: 2.7393
>      17280/200287 [=>............................] - ETA: 2:16 - loss: 2.7369
>      17408/200287 [=>............................] - ETA: 2:16 - loss: 2.7360
>      17536/200287 [=>............................] - ETA: 2:16 - loss: 2.7346
>      17664/200287 [=>............................] - ETA: 2:16 - loss: 2.7320
>      17792/200287 [=>............................] - ETA: 2:16 - loss: 2.7295
>      17920/200287 [=>............................] - ETA: 2:15 - loss: 2.7261
>      18048/200287 [=>............................] - ETA: 2:15 - loss: 2.7241
>      18176/200287 [=>............................] - ETA: 2:15 - loss: 2.7216
>      18304/200287 [=>............................] - ETA: 2:15 - loss: 2.7206
>      18432/200287 [=>............................] - ETA: 2:15 - loss: 2.7187
>      18560/200287 [=>............................] - ETA: 2:15 - loss: 2.7165
>      18688/200287 [=>............................] - ETA: 2:15 - loss: 2.7145
>      18816/200287 [=>............................] - ETA: 2:15 - loss: 2.7118
>      18944/200287 [=>............................] - ETA: 2:15 - loss: 2.7098
>      19072/200287 [=>............................] - ETA: 2:14 - loss: 2.7073
>      19200/200287 [=>............................] - ETA: 2:14 - loss: 2.7057
>      19328/200287 [=>............................] - ETA: 2:14 - loss: 2.7041
>      19456/200287 [=>............................] - ETA: 2:14 - loss: 2.7019
>      19584/200287 [=>............................] - ETA: 2:14 - loss: 2.7006
>      19712/200287 [=>............................] - ETA: 2:14 - loss: 2.6993
>      19840/200287 [=>............................] - ETA: 2:14 - loss: 2.6976
>      19968/200287 [=>............................] - ETA: 2:13 - loss: 2.6962
>      20096/200287 [==>...........................] - ETA: 2:13 - loss: 2.6933
>      20224/200287 [==>...........................] - ETA: 2:13 - loss: 2.6915
>      20352/200287 [==>...........................] - ETA: 2:13 - loss: 2.6904
>      20480/200287 [==>...........................] - ETA: 2:13 - loss: 2.6881
>      20608/200287 [==>...........................] - ETA: 2:13 - loss: 2.6859
>      20736/200287 [==>...........................] - ETA: 2:13 - loss: 2.6832
>      20864/200287 [==>...........................] - ETA: 2:13 - loss: 2.6809
>      20992/200287 [==>...........................] - ETA: 2:12 - loss: 2.6797
>      21120/200287 [==>...........................] - ETA: 2:12 - loss: 2.6787
>      21248/200287 [==>...........................] - ETA: 2:12 - loss: 2.6756
>      21376/200287 [==>...........................] - ETA: 2:12 - loss: 2.6730
>      21504/200287 [==>...........................] - ETA: 2:12 - loss: 2.6722
>      21632/200287 [==>...........................] - ETA: 2:12 - loss: 2.6700
>      21760/200287 [==>...........................] - ETA: 2:12 - loss: 2.6675
>      21888/200287 [==>...........................] - ETA: 2:12 - loss: 2.6656
>      22016/200287 [==>...........................] - ETA: 2:11 - loss: 2.6637
>      22144/200287 [==>...........................] - ETA: 2:11 - loss: 2.6611
>      22272/200287 [==>...........................] - ETA: 2:11 - loss: 2.6586
>      22400/200287 [==>...........................] - ETA: 2:11 - loss: 2.6573
>      22528/200287 [==>...........................] - ETA: 2:11 - loss: 2.6552
>      22656/200287 [==>...........................] - ETA: 2:11 - loss: 2.6544
>      22784/200287 [==>...........................] - ETA: 2:11 - loss: 2.6526
>      22912/200287 [==>...........................] - ETA: 2:11 - loss: 2.6509
>      23040/200287 [==>...........................] - ETA: 2:10 - loss: 2.6499
>      23168/200287 [==>...........................] - ETA: 2:10 - loss: 2.6476
>      23296/200287 [==>...........................] - ETA: 2:10 - loss: 2.6460
>      23424/200287 [==>...........................] - ETA: 2:10 - loss: 2.6440
>      23552/200287 [==>...........................] - ETA: 2:10 - loss: 2.6423
>      23680/200287 [==>...........................] - ETA: 2:10 - loss: 2.6393
>      23808/200287 [==>...........................] - ETA: 2:10 - loss: 2.6369
>      23936/200287 [==>...........................] - ETA: 2:10 - loss: 2.6363
>      24064/200287 [==>...........................] - ETA: 2:09 - loss: 2.6341
>      24192/200287 [==>...........................] - ETA: 2:09 - loss: 2.6324
>      24320/200287 [==>...........................] - ETA: 2:09 - loss: 2.6306
>      24448/200287 [==>...........................] - ETA: 2:09 - loss: 2.6284
>      24576/200287 [==>...........................] - ETA: 2:09 - loss: 2.6260
>      24704/200287 [==>...........................] - ETA: 2:09 - loss: 2.6237
>      24832/200287 [==>...........................] - ETA: 2:09 - loss: 2.6221
>      24960/200287 [==>...........................] - ETA: 2:09 - loss: 2.6197
>      25088/200287 [==>...........................] - ETA: 2:09 - loss: 2.6179
>      25216/200287 [==>...........................] - ETA: 2:08 - loss: 2.6154
>      25344/200287 [==>...........................] - ETA: 2:08 - loss: 2.6150
>      25472/200287 [==>...........................] - ETA: 2:08 - loss: 2.6136
>      25600/200287 [==>...........................] - ETA: 2:08 - loss: 2.6119
>      25728/200287 [==>...........................] - ETA: 2:08 - loss: 2.6103
>      25856/200287 [==>...........................] - ETA: 2:08 - loss: 2.6094
>      25984/200287 [==>...........................] - ETA: 2:08 - loss: 2.6074
>      26112/200287 [==>...........................] - ETA: 2:08 - loss: 2.6054
>      26240/200287 [==>...........................] - ETA: 2:07 - loss: 2.6039
>      26368/200287 [==>...........................] - ETA: 2:07 - loss: 2.6030
>      26496/200287 [==>...........................] - ETA: 2:07 - loss: 2.6014
>      26624/200287 [==>...........................] - ETA: 2:07 - loss: 2.5993
>      26752/200287 [===>..........................] - ETA: 2:07 - loss: 2.5974
>      26880/200287 [===>..........................] - ETA: 2:07 - loss: 2.5959
>      27008/200287 [===>..........................] - ETA: 2:07 - loss: 2.5947
>      27136/200287 [===>..........................] - ETA: 2:07 - loss: 2.5921
>      27264/200287 [===>..........................] - ETA: 2:07 - loss: 2.5904
>      27392/200287 [===>..........................] - ETA: 2:07 - loss: 2.5893
>      27520/200287 [===>..........................] - ETA: 2:06 - loss: 2.5878
>      27648/200287 [===>..........................] - ETA: 2:06 - loss: 2.5864
>      27776/200287 [===>..........................] - ETA: 2:06 - loss: 2.5849
>      27904/200287 [===>..........................] - ETA: 2:06 - loss: 2.5833
>      28032/200287 [===>..........................] - ETA: 2:06 - loss: 2.5819
>      28160/200287 [===>..........................] - ETA: 2:06 - loss: 2.5812
>      28288/200287 [===>..........................] - ETA: 2:06 - loss: 2.5791
>      28416/200287 [===>..........................] - ETA: 2:06 - loss: 2.5782
>      28544/200287 [===>..........................] - ETA: 2:05 - loss: 2.5767
>      28672/200287 [===>..........................] - ETA: 2:05 - loss: 2.5761
>      28800/200287 [===>..........................] - ETA: 2:05 - loss: 2.5745
>      28928/200287 [===>..........................] - ETA: 2:05 - loss: 2.5739
>      29056/200287 [===>..........................] - ETA: 2:05 - loss: 2.5727
>      29184/200287 [===>..........................] - ETA: 2:05 - loss: 2.5708
>      29312/200287 [===>..........................] - ETA: 2:05 - loss: 2.5690
>      29440/200287 [===>..........................] - ETA: 2:05 - loss: 2.5677
>      29568/200287 [===>..........................] - ETA: 2:05 - loss: 2.5664
>      29696/200287 [===>..........................] - ETA: 2:05 - loss: 2.5649
>      29824/200287 [===>..........................] - ETA: 2:04 - loss: 2.5632
>      29952/200287 [===>..........................] - ETA: 2:04 - loss: 2.5610
>      30080/200287 [===>..........................] - ETA: 2:04 - loss: 2.5586
>      30208/200287 [===>..........................] - ETA: 2:04 - loss: 2.5570
>      30336/200287 [===>..........................] - ETA: 2:04 - loss: 2.5560
>      30464/200287 [===>..........................] - ETA: 2:04 - loss: 2.5548
>      30592/200287 [===>..........................] - ETA: 2:04 - loss: 2.5536
>      30720/200287 [===>..........................] - ETA: 2:04 - loss: 2.5530
>      30848/200287 [===>..........................] - ETA: 2:04 - loss: 2.5520
>      30976/200287 [===>..........................] - ETA: 2:04 - loss: 2.5514
>      31104/200287 [===>..........................] - ETA: 2:04 - loss: 2.5500
>      31232/200287 [===>..........................] - ETA: 2:03 - loss: 2.5481
>      31360/200287 [===>..........................] - ETA: 2:03 - loss: 2.5464
>      31488/200287 [===>..........................] - ETA: 2:03 - loss: 2.5459
>      31616/200287 [===>..........................] - ETA: 2:03 - loss: 2.5444
>      31744/200287 [===>..........................] - ETA: 2:03 - loss: 2.5429
>      31872/200287 [===>..........................] - ETA: 2:03 - loss: 2.5422
>      32000/200287 [===>..........................] - ETA: 2:03 - loss: 2.5411
>      32128/200287 [===>..........................] - ETA: 2:03 - loss: 2.5398
>      32256/200287 [===>..........................] - ETA: 2:03 - loss: 2.5389
>      32384/200287 [===>..........................] - ETA: 2:02 - loss: 2.5378
>      32512/200287 [===>..........................] - ETA: 2:02 - loss: 2.5369
>      32640/200287 [===>..........................] - ETA: 2:02 - loss: 2.5359
>      32768/200287 [===>..........................] - ETA: 2:02 - loss: 2.5348
>      32896/200287 [===>..........................] - ETA: 2:02 - loss: 2.5330
>      33024/200287 [===>..........................] - ETA: 2:02 - loss: 2.5329
>      33152/200287 [===>..........................] - ETA: 2:02 - loss: 2.5310
>      33280/200287 [===>..........................] - ETA: 2:02 - loss: 2.5300
>      33408/200287 [====>.........................] - ETA: 2:02 - loss: 2.5287
>      33536/200287 [====>.........................] - ETA: 2:02 - loss: 2.5276
>      33664/200287 [====>.........................] - ETA: 2:01 - loss: 2.5262
>      33792/200287 [====>.........................] - ETA: 2:01 - loss: 2.5252
>      33920/200287 [====>.........................] - ETA: 2:01 - loss: 2.5244
>      34048/200287 [====>.........................] - ETA: 2:01 - loss: 2.5224
>      34176/200287 [====>.........................] - ETA: 2:01 - loss: 2.5210
>      34304/200287 [====>.........................] - ETA: 2:01 - loss: 2.5202
>      34432/200287 [====>.........................] - ETA: 2:01 - loss: 2.5188
>      34560/200287 [====>.........................] - ETA: 2:01 - loss: 2.5177
>      34688/200287 [====>.........................] - ETA: 2:01 - loss: 2.5168
>      34816/200287 [====>.........................] - ETA: 2:01 - loss: 2.5157
>      34944/200287 [====>.........................] - ETA: 2:00 - loss: 2.5145
>      35072/200287 [====>.........................] - ETA: 2:00 - loss: 2.5129
>      35200/200287 [====>.........................] - ETA: 2:00 - loss: 2.5127
>      35328/200287 [====>.........................] - ETA: 2:00 - loss: 2.5121
>      35456/200287 [====>.........................] - ETA: 2:00 - loss: 2.5100
>      35584/200287 [====>.........................] - ETA: 2:00 - loss: 2.5094
>      35712/200287 [====>.........................] - ETA: 2:00 - loss: 2.5083
>      35840/200287 [====>.........................] - ETA: 2:00 - loss: 2.5071
>      35968/200287 [====>.........................] - ETA: 2:00 - loss: 2.5057
>      36096/200287 [====>.........................] - ETA: 1:59 - loss: 2.5047
>      36224/200287 [====>.........................] - ETA: 1:59 - loss: 2.5037
>      36352/200287 [====>.........................] - ETA: 1:59 - loss: 2.5024
>      36480/200287 [====>.........................] - ETA: 1:59 - loss: 2.5013
>      36608/200287 [====>.........................] - ETA: 1:59 - loss: 2.5003
>      36736/200287 [====>.........................] - ETA: 1:59 - loss: 2.4993
>      36864/200287 [====>.........................] - ETA: 1:59 - loss: 2.4978
>      36992/200287 [====>.........................] - ETA: 1:59 - loss: 2.4973
>      37120/200287 [====>.........................] - ETA: 1:59 - loss: 2.4961
>      37248/200287 [====>.........................] - ETA: 1:59 - loss: 2.4950
>      37376/200287 [====>.........................] - ETA: 1:58 - loss: 2.4938
>      37504/200287 [====>.........................] - ETA: 1:58 - loss: 2.4924
>      37632/200287 [====>.........................] - ETA: 1:58 - loss: 2.4913
>      37760/200287 [====>.........................] - ETA: 1:58 - loss: 2.4901
>      37888/200287 [====>.........................] - ETA: 1:58 - loss: 2.4892
>      38016/200287 [====>.........................] - ETA: 1:58 - loss: 2.4883
>      38144/200287 [====>.........................] - ETA: 1:58 - loss: 2.4876
>      38272/200287 [====>.........................] - ETA: 1:58 - loss: 2.4865
>      38400/200287 [====>.........................] - ETA: 1:58 - loss: 2.4854
>      38528/200287 [====>.........................] - ETA: 1:58 - loss: 2.4849
>      38656/200287 [====>.........................] - ETA: 1:57 - loss: 2.4842
>      38784/200287 [====>.........................] - ETA: 1:57 - loss: 2.4837
>      38912/200287 [====>.........................] - ETA: 1:57 - loss: 2.4828
>      39040/200287 [====>.........................] - ETA: 1:57 - loss: 2.4816
>      39168/200287 [====>.........................] - ETA: 1:57 - loss: 2.4808
>      39296/200287 [====>.........................] - ETA: 1:57 - loss: 2.4802
>      39424/200287 [====>.........................] - ETA: 1:57 - loss: 2.4790
>      39552/200287 [====>.........................] - ETA: 1:57 - loss: 2.4777
>      39680/200287 [====>.........................] - ETA: 1:57 - loss: 2.4756
>      39808/200287 [====>.........................] - ETA: 1:57 - loss: 2.4746
>      39936/200287 [====>.........................] - ETA: 1:56 - loss: 2.4740
>      40064/200287 [=====>........................] - ETA: 1:56 - loss: 2.4724
>      40192/200287 [=====>........................] - ETA: 1:56 - loss: 2.4719
>      40320/200287 [=====>........................] - ETA: 1:56 - loss: 2.4706
>      40448/200287 [=====>........................] - ETA: 1:56 - loss: 2.4697
>      40576/200287 [=====>........................] - ETA: 1:56 - loss: 2.4689
>      40704/200287 [=====>........................] - ETA: 1:56 - loss: 2.4686
>      40832/200287 [=====>........................] - ETA: 1:56 - loss: 2.4675
>      40960/200287 [=====>........................] - ETA: 1:56 - loss: 2.4664
>      41088/200287 [=====>........................] - ETA: 1:56 - loss: 2.4655
>
>     *** WARNING: skipped 6400810 bytes of output ***
>
>     160128/200287 [======================>.......] - ETA: 28s - loss: 1.2923
>     160256/200287 [=======================>......] - ETA: 28s - loss: 1.2924
>     160384/200287 [=======================>......] - ETA: 28s - loss: 1.2924
>     160512/200287 [=======================>......] - ETA: 27s - loss: 1.2927
>     160640/200287 [=======================>......] - ETA: 27s - loss: 1.2927
>     160768/200287 [=======================>......] - ETA: 27s - loss: 1.2925
>     160896/200287 [=======================>......] - ETA: 27s - loss: 1.2926
>     161024/200287 [=======================>......] - ETA: 27s - loss: 1.2926
>     161152/200287 [=======================>......] - ETA: 27s - loss: 1.2926
>     161280/200287 [=======================>......] - ETA: 27s - loss: 1.2926
>     161408/200287 [=======================>......] - ETA: 27s - loss: 1.2924
>     161536/200287 [=======================>......] - ETA: 27s - loss: 1.2925
>     161664/200287 [=======================>......] - ETA: 27s - loss: 1.2924
>     161792/200287 [=======================>......] - ETA: 27s - loss: 1.2924
>     161920/200287 [=======================>......] - ETA: 27s - loss: 1.2924
>     162048/200287 [=======================>......] - ETA: 26s - loss: 1.2925
>     162176/200287 [=======================>......] - ETA: 26s - loss: 1.2925
>     162304/200287 [=======================>......] - ETA: 26s - loss: 1.2924
>     162432/200287 [=======================>......] - ETA: 26s - loss: 1.2924
>     162560/200287 [=======================>......] - ETA: 26s - loss: 1.2923
>     162688/200287 [=======================>......] - ETA: 26s - loss: 1.2924
>     162816/200287 [=======================>......] - ETA: 26s - loss: 1.2922
>     162944/200287 [=======================>......] - ETA: 26s - loss: 1.2921
>     163072/200287 [=======================>......] - ETA: 26s - loss: 1.2922
>     163200/200287 [=======================>......] - ETA: 26s - loss: 1.2921
>     163328/200287 [=======================>......] - ETA: 26s - loss: 1.2919
>     163456/200287 [=======================>......] - ETA: 25s - loss: 1.2921
>     163584/200287 [=======================>......] - ETA: 25s - loss: 1.2920
>     163712/200287 [=======================>......] - ETA: 25s - loss: 1.2920
>     163840/200287 [=======================>......] - ETA: 25s - loss: 1.2921
>     163968/200287 [=======================>......] - ETA: 25s - loss: 1.2922
>     164096/200287 [=======================>......] - ETA: 25s - loss: 1.2922
>     164224/200287 [=======================>......] - ETA: 25s - loss: 1.2922
>     164352/200287 [=======================>......] - ETA: 25s - loss: 1.2923
>     164480/200287 [=======================>......] - ETA: 25s - loss: 1.2925
>     164608/200287 [=======================>......] - ETA: 25s - loss: 1.2927
>     164736/200287 [=======================>......] - ETA: 25s - loss: 1.2929
>     164864/200287 [=======================>......] - ETA: 24s - loss: 1.2928
>     164992/200287 [=======================>......] - ETA: 24s - loss: 1.2927
>     165120/200287 [=======================>......] - ETA: 24s - loss: 1.2927
>     165248/200287 [=======================>......] - ETA: 24s - loss: 1.2927
>     165376/200287 [=======================>......] - ETA: 24s - loss: 1.2927
>     165504/200287 [=======================>......] - ETA: 24s - loss: 1.2928
>     165632/200287 [=======================>......] - ETA: 24s - loss: 1.2928
>     165760/200287 [=======================>......] - ETA: 24s - loss: 1.2927
>     165888/200287 [=======================>......] - ETA: 24s - loss: 1.2927
>     166016/200287 [=======================>......] - ETA: 24s - loss: 1.2928
>     166144/200287 [=======================>......] - ETA: 24s - loss: 1.2929
>     166272/200287 [=======================>......] - ETA: 23s - loss: 1.2930
>     166400/200287 [=======================>......] - ETA: 23s - loss: 1.2931
>     166528/200287 [=======================>......] - ETA: 23s - loss: 1.2932
>     166656/200287 [=======================>......] - ETA: 23s - loss: 1.2932
>     166784/200287 [=======================>......] - ETA: 23s - loss: 1.2934
>     166912/200287 [========================>.....] - ETA: 23s - loss: 1.2932
>     167040/200287 [========================>.....] - ETA: 23s - loss: 1.2932
>     167168/200287 [========================>.....] - ETA: 23s - loss: 1.2932
>     167296/200287 [========================>.....] - ETA: 23s - loss: 1.2932
>     167424/200287 [========================>.....] - ETA: 23s - loss: 1.2932
>     167552/200287 [========================>.....] - ETA: 23s - loss: 1.2933
>     167680/200287 [========================>.....] - ETA: 22s - loss: 1.2932
>     167808/200287 [========================>.....] - ETA: 22s - loss: 1.2933
>     167936/200287 [========================>.....] - ETA: 22s - loss: 1.2934
>     168064/200287 [========================>.....] - ETA: 22s - loss: 1.2934
>     168192/200287 [========================>.....] - ETA: 22s - loss: 1.2935
>     168320/200287 [========================>.....] - ETA: 22s - loss: 1.2934
>     168448/200287 [========================>.....] - ETA: 22s - loss: 1.2933
>     168576/200287 [========================>.....] - ETA: 22s - loss: 1.2935
>     168704/200287 [========================>.....] - ETA: 22s - loss: 1.2936
>     168832/200287 [========================>.....] - ETA: 22s - loss: 1.2937
>     168960/200287 [========================>.....] - ETA: 22s - loss: 1.2939
>     169088/200287 [========================>.....] - ETA: 21s - loss: 1.2938
>     169216/200287 [========================>.....] - ETA: 21s - loss: 1.2937
>     169344/200287 [========================>.....] - ETA: 21s - loss: 1.2938
>     169472/200287 [========================>.....] - ETA: 21s - loss: 1.2938
>     169600/200287 [========================>.....] - ETA: 21s - loss: 1.2937
>     169728/200287 [========================>.....] - ETA: 21s - loss: 1.2936
>     169856/200287 [========================>.....] - ETA: 21s - loss: 1.2938
>     169984/200287 [========================>.....] - ETA: 21s - loss: 1.2938
>     170112/200287 [========================>.....] - ETA: 21s - loss: 1.2938
>     170240/200287 [========================>.....] - ETA: 21s - loss: 1.2937
>     170368/200287 [========================>.....] - ETA: 21s - loss: 1.2937
>     170496/200287 [========================>.....] - ETA: 20s - loss: 1.2936
>     170624/200287 [========================>.....] - ETA: 20s - loss: 1.2938
>     170752/200287 [========================>.....] - ETA: 20s - loss: 1.2937
>     170880/200287 [========================>.....] - ETA: 20s - loss: 1.2937
>     171008/200287 [========================>.....] - ETA: 20s - loss: 1.2936
>     171136/200287 [========================>.....] - ETA: 20s - loss: 1.2937
>     171264/200287 [========================>.....] - ETA: 20s - loss: 1.2936
>     171392/200287 [========================>.....] - ETA: 20s - loss: 1.2936
>     171520/200287 [========================>.....] - ETA: 20s - loss: 1.2935
>     171648/200287 [========================>.....] - ETA: 20s - loss: 1.2934
>     171776/200287 [========================>.....] - ETA: 20s - loss: 1.2934
>     171904/200287 [========================>.....] - ETA: 19s - loss: 1.2934
>     172032/200287 [========================>.....] - ETA: 19s - loss: 1.2934
>     172160/200287 [========================>.....] - ETA: 19s - loss: 1.2933
>     172288/200287 [========================>.....] - ETA: 19s - loss: 1.2931
>     172416/200287 [========================>.....] - ETA: 19s - loss: 1.2930
>     172544/200287 [========================>.....] - ETA: 19s - loss: 1.2930
>     172672/200287 [========================>.....] - ETA: 19s - loss: 1.2929
>     172800/200287 [========================>.....] - ETA: 19s - loss: 1.2930
>     172928/200287 [========================>.....] - ETA: 19s - loss: 1.2930
>     173056/200287 [========================>.....] - ETA: 19s - loss: 1.2931
>     173184/200287 [========================>.....] - ETA: 19s - loss: 1.2930
>     173312/200287 [========================>.....] - ETA: 18s - loss: 1.2930
>     173440/200287 [========================>.....] - ETA: 18s - loss: 1.2929
>     173568/200287 [========================>.....] - ETA: 18s - loss: 1.2929
>     173696/200287 [=========================>....] - ETA: 18s - loss: 1.2929
>     173824/200287 [=========================>....] - ETA: 18s - loss: 1.2928
>     173952/200287 [=========================>....] - ETA: 18s - loss: 1.2928
>     174080/200287 [=========================>....] - ETA: 18s - loss: 1.2928
>     174208/200287 [=========================>....] - ETA: 18s - loss: 1.2928
>     174336/200287 [=========================>....] - ETA: 18s - loss: 1.2930
>     174464/200287 [=========================>....] - ETA: 18s - loss: 1.2929
>     174592/200287 [=========================>....] - ETA: 18s - loss: 1.2931
>     174720/200287 [=========================>....] - ETA: 17s - loss: 1.2932
>     174848/200287 [=========================>....] - ETA: 17s - loss: 1.2933
>     174976/200287 [=========================>....] - ETA: 17s - loss: 1.2932
>     175104/200287 [=========================>....] - ETA: 17s - loss: 1.2932
>     175232/200287 [=========================>....] - ETA: 17s - loss: 1.2932
>     175360/200287 [=========================>....] - ETA: 17s - loss: 1.2931
>     175488/200287 [=========================>....] - ETA: 17s - loss: 1.2931
>     175616/200287 [=========================>....] - ETA: 17s - loss: 1.2930
>     175744/200287 [=========================>....] - ETA: 17s - loss: 1.2930
>     175872/200287 [=========================>....] - ETA: 17s - loss: 1.2931
>     176000/200287 [=========================>....] - ETA: 17s - loss: 1.2933
>     176128/200287 [=========================>....] - ETA: 16s - loss: 1.2935
>     176256/200287 [=========================>....] - ETA: 16s - loss: 1.2935
>     176384/200287 [=========================>....] - ETA: 16s - loss: 1.2935
>     176512/200287 [=========================>....] - ETA: 16s - loss: 1.2934
>     176640/200287 [=========================>....] - ETA: 16s - loss: 1.2937
>     176768/200287 [=========================>....] - ETA: 16s - loss: 1.2937
>     176896/200287 [=========================>....] - ETA: 16s - loss: 1.2936
>     177024/200287 [=========================>....] - ETA: 16s - loss: 1.2936
>     177152/200287 [=========================>....] - ETA: 16s - loss: 1.2936
>     177280/200287 [=========================>....] - ETA: 16s - loss: 1.2936
>     177408/200287 [=========================>....] - ETA: 16s - loss: 1.2936
>     177536/200287 [=========================>....] - ETA: 16s - loss: 1.2934
>     177664/200287 [=========================>....] - ETA: 15s - loss: 1.2934
>     177792/200287 [=========================>....] - ETA: 15s - loss: 1.2935
>     177920/200287 [=========================>....] - ETA: 15s - loss: 1.2935
>     178048/200287 [=========================>....] - ETA: 15s - loss: 1.2936
>     178176/200287 [=========================>....] - ETA: 15s - loss: 1.2935
>     178304/200287 [=========================>....] - ETA: 15s - loss: 1.2936
>     178432/200287 [=========================>....] - ETA: 15s - loss: 1.2936
>     178560/200287 [=========================>....] - ETA: 15s - loss: 1.2935
>     178688/200287 [=========================>....] - ETA: 15s - loss: 1.2933
>     178816/200287 [=========================>....] - ETA: 15s - loss: 1.2932
>     178944/200287 [=========================>....] - ETA: 15s - loss: 1.2931
>     179072/200287 [=========================>....] - ETA: 14s - loss: 1.2930
>     179200/200287 [=========================>....] - ETA: 14s - loss: 1.2930
>     179328/200287 [=========================>....] - ETA: 14s - loss: 1.2931
>     179456/200287 [=========================>....] - ETA: 14s - loss: 1.2932
>     179584/200287 [=========================>....] - ETA: 14s - loss: 1.2932
>     179712/200287 [=========================>....] - ETA: 14s - loss: 1.2932
>     179840/200287 [=========================>....] - ETA: 14s - loss: 1.2932
>     179968/200287 [=========================>....] - ETA: 14s - loss: 1.2933
>     180096/200287 [=========================>....] - ETA: 14s - loss: 1.2934
>     180224/200287 [=========================>....] - ETA: 14s - loss: 1.2934
>     180352/200287 [==========================>...] - ETA: 14s - loss: 1.2934
>     180480/200287 [==========================>...] - ETA: 13s - loss: 1.2933
>     180608/200287 [==========================>...] - ETA: 13s - loss: 1.2933
>     180736/200287 [==========================>...] - ETA: 13s - loss: 1.2933
>     180864/200287 [==========================>...] - ETA: 13s - loss: 1.2933
>     180992/200287 [==========================>...] - ETA: 13s - loss: 1.2933
>     181120/200287 [==========================>...] - ETA: 13s - loss: 1.2932
>     181248/200287 [==========================>...] - ETA: 13s - loss: 1.2933
>     181376/200287 [==========================>...] - ETA: 13s - loss: 1.2933
>     181504/200287 [==========================>...] - ETA: 13s - loss: 1.2932
>     181632/200287 [==========================>...] - ETA: 13s - loss: 1.2932
>     181760/200287 [==========================>...] - ETA: 13s - loss: 1.2933
>     181888/200287 [==========================>...] - ETA: 12s - loss: 1.2933
>     182016/200287 [==========================>...] - ETA: 12s - loss: 1.2933
>     182144/200287 [==========================>...] - ETA: 12s - loss: 1.2933
>     182272/200287 [==========================>...] - ETA: 12s - loss: 1.2934
>     182400/200287 [==========================>...] - ETA: 12s - loss: 1.2933
>     182528/200287 [==========================>...] - ETA: 12s - loss: 1.2934
>     182656/200287 [==========================>...] - ETA: 12s - loss: 1.2934
>     182784/200287 [==========================>...] - ETA: 12s - loss: 1.2933
>     182912/200287 [==========================>...] - ETA: 12s - loss: 1.2933
>     183040/200287 [==========================>...] - ETA: 12s - loss: 1.2934
>     183168/200287 [==========================>...] - ETA: 12s - loss: 1.2934
>     183296/200287 [==========================>...] - ETA: 11s - loss: 1.2934
>     183424/200287 [==========================>...] - ETA: 11s - loss: 1.2936
>     183552/200287 [==========================>...] - ETA: 11s - loss: 1.2935
>     183680/200287 [==========================>...] - ETA: 11s - loss: 1.2938
>     183808/200287 [==========================>...] - ETA: 11s - loss: 1.2937
>     183936/200287 [==========================>...] - ETA: 11s - loss: 1.2937
>     184064/200287 [==========================>...] - ETA: 11s - loss: 1.2935
>     184192/200287 [==========================>...] - ETA: 11s - loss: 1.2937
>     184320/200287 [==========================>...] - ETA: 11s - loss: 1.2935
>     184448/200287 [==========================>...] - ETA: 11s - loss: 1.2937
>     184576/200287 [==========================>...] - ETA: 11s - loss: 1.2936
>     184704/200287 [==========================>...] - ETA: 10s - loss: 1.2938
>     184832/200287 [==========================>...] - ETA: 10s - loss: 1.2937
>     184960/200287 [==========================>...] - ETA: 10s - loss: 1.2938
>     185088/200287 [==========================>...] - ETA: 10s - loss: 1.2939
>     185216/200287 [==========================>...] - ETA: 10s - loss: 1.2939
>     185344/200287 [==========================>...] - ETA: 10s - loss: 1.2938
>     185472/200287 [==========================>...] - ETA: 10s - loss: 1.2938
>     185600/200287 [==========================>...] - ETA: 10s - loss: 1.2940
>     185728/200287 [==========================>...] - ETA: 10s - loss: 1.2940
>     185856/200287 [==========================>...] - ETA: 10s - loss: 1.2940
>     185984/200287 [==========================>...] - ETA: 10s - loss: 1.2939
>     186112/200287 [==========================>...] - ETA: 9s - loss: 1.2939 
>     186240/200287 [==========================>...] - ETA: 9s - loss: 1.2940
>     186368/200287 [==========================>...] - ETA: 9s - loss: 1.2940
>     186496/200287 [==========================>...] - ETA: 9s - loss: 1.2940
>     186624/200287 [==========================>...] - ETA: 9s - loss: 1.2941
>     186752/200287 [==========================>...] - ETA: 9s - loss: 1.2940
>     186880/200287 [==========================>...] - ETA: 9s - loss: 1.2940
>     187008/200287 [===========================>..] - ETA: 9s - loss: 1.2938
>     187136/200287 [===========================>..] - ETA: 9s - loss: 1.2937
>     187264/200287 [===========================>..] - ETA: 9s - loss: 1.2937
>     187392/200287 [===========================>..] - ETA: 9s - loss: 1.2936
>     187520/200287 [===========================>..] - ETA: 8s - loss: 1.2936
>     187648/200287 [===========================>..] - ETA: 8s - loss: 1.2936
>     187776/200287 [===========================>..] - ETA: 8s - loss: 1.2939
>     187904/200287 [===========================>..] - ETA: 8s - loss: 1.2939
>     188032/200287 [===========================>..] - ETA: 8s - loss: 1.2938
>     188160/200287 [===========================>..] - ETA: 8s - loss: 1.2937
>     188288/200287 [===========================>..] - ETA: 8s - loss: 1.2937
>     188416/200287 [===========================>..] - ETA: 8s - loss: 1.2938
>     188544/200287 [===========================>..] - ETA: 8s - loss: 1.2938
>     188672/200287 [===========================>..] - ETA: 8s - loss: 1.2937
>     188800/200287 [===========================>..] - ETA: 8s - loss: 1.2937
>     188928/200287 [===========================>..] - ETA: 7s - loss: 1.2935
>     189056/200287 [===========================>..] - ETA: 7s - loss: 1.2935
>     189184/200287 [===========================>..] - ETA: 7s - loss: 1.2936
>     189312/200287 [===========================>..] - ETA: 7s - loss: 1.2935
>     189440/200287 [===========================>..] - ETA: 7s - loss: 1.2935
>     189568/200287 [===========================>..] - ETA: 7s - loss: 1.2934
>     189696/200287 [===========================>..] - ETA: 7s - loss: 1.2934
>     189824/200287 [===========================>..] - ETA: 7s - loss: 1.2934
>     189952/200287 [===========================>..] - ETA: 7s - loss: 1.2935
>     190080/200287 [===========================>..] - ETA: 7s - loss: 1.2936
>     190208/200287 [===========================>..] - ETA: 7s - loss: 1.2937
>     190336/200287 [===========================>..] - ETA: 7s - loss: 1.2936
>     190464/200287 [===========================>..] - ETA: 6s - loss: 1.2934
>     190592/200287 [===========================>..] - ETA: 6s - loss: 1.2936
>     190720/200287 [===========================>..] - ETA: 6s - loss: 1.2936
>     190848/200287 [===========================>..] - ETA: 6s - loss: 1.2936
>     190976/200287 [===========================>..] - ETA: 6s - loss: 1.2935
>     191104/200287 [===========================>..] - ETA: 6s - loss: 1.2936
>     191232/200287 [===========================>..] - ETA: 6s - loss: 1.2937
>     191360/200287 [===========================>..] - ETA: 6s - loss: 1.2937
>     191488/200287 [===========================>..] - ETA: 6s - loss: 1.2937
>     191616/200287 [===========================>..] - ETA: 6s - loss: 1.2938
>     191744/200287 [===========================>..] - ETA: 6s - loss: 1.2938
>     191872/200287 [===========================>..] - ETA: 5s - loss: 1.2939
>     192000/200287 [===========================>..] - ETA: 5s - loss: 1.2938
>     192128/200287 [===========================>..] - ETA: 5s - loss: 1.2939
>     192256/200287 [===========================>..] - ETA: 5s - loss: 1.2939
>     192384/200287 [===========================>..] - ETA: 5s - loss: 1.2942
>     192512/200287 [===========================>..] - ETA: 5s - loss: 1.2942
>     192640/200287 [===========================>..] - ETA: 5s - loss: 1.2942
>     192768/200287 [===========================>..] - ETA: 5s - loss: 1.2941
>     192896/200287 [===========================>..] - ETA: 5s - loss: 1.2940
>     193024/200287 [===========================>..] - ETA: 5s - loss: 1.2941
>     193152/200287 [===========================>..] - ETA: 5s - loss: 1.2940
>     193280/200287 [===========================>..] - ETA: 4s - loss: 1.2942
>     193408/200287 [===========================>..] - ETA: 4s - loss: 1.2942
>     193536/200287 [===========================>..] - ETA: 4s - loss: 1.2942
>     193664/200287 [============================>.] - ETA: 4s - loss: 1.2943
>     193792/200287 [============================>.] - ETA: 4s - loss: 1.2944
>     193920/200287 [============================>.] - ETA: 4s - loss: 1.2944
>     194048/200287 [============================>.] - ETA: 4s - loss: 1.2944
>     194176/200287 [============================>.] - ETA: 4s - loss: 1.2944
>     194304/200287 [============================>.] - ETA: 4s - loss: 1.2945
>     194432/200287 [============================>.] - ETA: 4s - loss: 1.2945
>     194560/200287 [============================>.] - ETA: 4s - loss: 1.2945
>     194688/200287 [============================>.] - ETA: 3s - loss: 1.2945
>     194816/200287 [============================>.] - ETA: 3s - loss: 1.2946
>     194944/200287 [============================>.] - ETA: 3s - loss: 1.2945
>     195072/200287 [============================>.] - ETA: 3s - loss: 1.2945
>     195200/200287 [============================>.] - ETA: 3s - loss: 1.2946
>     195328/200287 [============================>.] - ETA: 3s - loss: 1.2945
>     195456/200287 [============================>.] - ETA: 3s - loss: 1.2946
>     195584/200287 [============================>.] - ETA: 3s - loss: 1.2947
>     195712/200287 [============================>.] - ETA: 3s - loss: 1.2946
>     195840/200287 [============================>.] - ETA: 3s - loss: 1.2945
>     195968/200287 [============================>.] - ETA: 3s - loss: 1.2945
>     196096/200287 [============================>.] - ETA: 2s - loss: 1.2944
>     196224/200287 [============================>.] - ETA: 2s - loss: 1.2945
>     196352/200287 [============================>.] - ETA: 2s - loss: 1.2944
>     196480/200287 [============================>.] - ETA: 2s - loss: 1.2945
>     196608/200287 [============================>.] - ETA: 2s - loss: 1.2946
>     196736/200287 [============================>.] - ETA: 2s - loss: 1.2946
>     196864/200287 [============================>.] - ETA: 2s - loss: 1.2945
>     196992/200287 [============================>.] - ETA: 2s - loss: 1.2946
>     197120/200287 [============================>.] - ETA: 2s - loss: 1.2945
>     197248/200287 [============================>.] - ETA: 2s - loss: 1.2946
>     197376/200287 [============================>.] - ETA: 2s - loss: 1.2947
>     197504/200287 [============================>.] - ETA: 1s - loss: 1.2947
>     197632/200287 [============================>.] - ETA: 1s - loss: 1.2949
>     197760/200287 [============================>.] - ETA: 1s - loss: 1.2950
>     197888/200287 [============================>.] - ETA: 1s - loss: 1.2950
>     198016/200287 [============================>.] - ETA: 1s - loss: 1.2950
>     198144/200287 [============================>.] - ETA: 1s - loss: 1.2950
>     198272/200287 [============================>.] - ETA: 1s - loss: 1.2950
>     198400/200287 [============================>.] - ETA: 1s - loss: 1.2950
>     198528/200287 [============================>.] - ETA: 1s - loss: 1.2950
>     198656/200287 [============================>.] - ETA: 1s - loss: 1.2951
>     198784/200287 [============================>.] - ETA: 1s - loss: 1.2951
>     198912/200287 [============================>.] - ETA: 0s - loss: 1.2952
>     199040/200287 [============================>.] - ETA: 0s - loss: 1.2952
>     199168/200287 [============================>.] - ETA: 0s - loss: 1.2952
>     199296/200287 [============================>.] - ETA: 0s - loss: 1.2954
>     199424/200287 [============================>.] - ETA: 0s - loss: 1.2955
>     199552/200287 [============================>.] - ETA: 0s - loss: 1.2955
>     199680/200287 [============================>.] - ETA: 0s - loss: 1.2956
>     199808/200287 [============================>.] - ETA: 0s - loss: 1.2955
>     199936/200287 [============================>.] - ETA: 0s - loss: 1.2957
>     200064/200287 [============================>.] - ETA: 0s - loss: 1.2958
>     200192/200287 [============================>.] - ETA: 0s - loss: 1.2958
>     200287/200287 [==============================] - 141s 704us/step - loss: 1.2958
>
>     ----- diversity: 0.2
>     ----- Generating with seed: "e things exist
>     (although in point of fac"
>     e things exist
>     (although in point of fact that the last in the same reamity of the same philosophy and the streen and the science of the most sense and actions and the primitive the soul of the problem of the protection of the spirit of the process of the soul of the same mankind, and the strange of the process to the strange, and with the strength of the soul and the streen and all the same reaming the strength of the contemplation of 
>
>     ----- diversity: 0.5
>     ----- Generating with seed: "e things exist
>     (although in point of fac"
>     e things exist
>     (although in point of fact against
>     the fundamental manking and the lapses in a thought to see through the same rumpotive to an endeaves and the propestible, and a soul of the most case of superiors, and a constitution of the same and strength and are necessary and that every schopenhauerd of the books and deceived to the designative, process always body and sympathy of the nature with another be in the strange and into th
>
>     ----- diversity: 1.0
>     ----- Generating with seed: "e things exist
>     (although in point of fac"
>     e things exist
>     (although in point of faceoric there under tyrannian
>     to its
>     ardt
>     hearts anying one of soul yet it is affinie, we might are
>     scheepjessed and if to begin has enwalled number certain se subtlest neighbory much is inligios of ;     scloymis of deceion nederate and plency of the feeling of the historily causes: one must suscreds, to
>     weary on
>     masture
>     is circumstance of the race
>     simply andumanily, fly was anything about to prion
>
>     ----- diversity: 1.2
>     ----- Generating with seed: "e things exist
>     (although in point of fac"
>     e things exist
>     (although in point of fact all
>     powers--rule about to dejess set
>     and ibbreond of the
>     chiralitical
>     and tapabfecician.
>
>
>     1w him the time one may have togal tpil
>     againist
>     mankind. even of him; at the prirfom presenth by german that the crual blice of bort to able that nowadays  iheat, with otherso indo if theh. perhaps an
>     power.
>
>     o